In [1]:
import deepspeed
import torch
import torch.nn as nn
from datasets import load_dataset
from deepspeed import PipelineModule
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers.models.gpt2.modeling_gpt2 import GPT2Block as GPT2BlockBase
import torch.distributed as dist

from transformers import DataCollatorWithPadding, get_scheduler

[2024-04-17 22:55:12,774] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-04-17 22:55:12,882] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-devel package with yum
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/usr/bin/ld: cannot find -laio
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.1
 [WARNING]  using untested triton version (2.1.0), only 1.0.0 is known to be compatible


In [2]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [3]:
batch_size, train_steps = 16, 300
train_samples = batch_size * train_steps

datasets = load_dataset("squad").data["train"]["context"]
datasets = [str(sample) for i, sample in enumerate(datasets)]
datasets = [
        tokenizer(data, return_tensors="pt", max_length=1024).input_ids[0]
        for data in tqdm(datasets)
]

100%|██████████| 87599/87599 [00:50<00:00, 1729.26it/s]


In [4]:
type(datasets)

list

In [5]:
print(datasets[:2])

[tensor([19895,  5712, 20221,    11,   262,  1524,   468,   257,  7835,  2095,
           13,  1629,   404,   262,  8774, 11819,   338,  3869, 29500,   318,
          257, 10861, 15207,   286,   262,  5283,  5335,    13, 34528,   287,
         2166,   286,   262,  8774, 11819,   290,  6476,   340,    11,   318,
          257, 15317, 15207,   286,  1951,   351,  5101,   510, 49309,   351,
          262,  8177,   366, 37522,   578,  1215,  2185, 16543,  2516,  1911,
         7406,   284,   262,  8774, 11819,   318,   262, 32520,  3970,   286,
          262, 17380,  8894,    13, 34528,  2157,   262, 37792,  3970,   318,
          262, 10299, 33955,    11,   257, 37919,  1295,   286, 11443,   290,
        14580,    13,   632,   318,   257, 30069,   286,   262,  7128, 33955,
          379,   406,   454,  8906,    11,  4881,   810,   262,  5283,  5335,
         1128,  7241,   306,  4120,   284,  9281,  6206,   324,  5857,   311,
        12944,   343,   516,   287,  1248,  3365,    13,  1629,

In [6]:
len(datasets)

87599

In [7]:
def collate_fn(batch):
    batch_encoding = tokenizer.pad(
        {"input_ids": batch}, padding="max_length", max_length=1024
    )
    return batch_encoding.input_ids


data_loader = iter (
         DataLoader(
            sorted(datasets, key=len, reverse=True),
            # uniform length batching
            # https://mccormickml.com/2020/07/29/smart-batching-tutorial/
            batch_size=batch_size,
            num_workers=8,
            collate_fn=collate_fn,
            shuffle=False,
       )
    )

len(data_loader)

5475

In [8]:
datasets = load_dataset("squad").data["train"]["context"]
datasets = [str(sample) for i, sample in enumerate(datasets) if i < train_samples]
#datasets = [
#        tokenizer(data, return_tensors="pt", max_length=1024).input_ids[0]
#        for data in tqdm(datasets)
#]

In [9]:
sample_dataset = datasets[:2]
sample_dataset

['Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building

In [10]:
token = [tokenizer(data).input_ids for data in sample_dataset]
print(token)

[[19895, 5712, 20221, 11, 262, 1524, 468, 257, 7835, 2095, 13, 1629, 404, 262, 8774, 11819, 338, 3869, 29500, 318, 257, 10861, 15207, 286, 262, 5283, 5335, 13, 34528, 287, 2166, 286, 262, 8774, 11819, 290, 6476, 340, 11, 318, 257, 15317, 15207, 286, 1951, 351, 5101, 510, 49309, 351, 262, 8177, 366, 37522, 578, 1215, 2185, 16543, 2516, 1911, 7406, 284, 262, 8774, 11819, 318, 262, 32520, 3970, 286, 262, 17380, 8894, 13, 34528, 2157, 262, 37792, 3970, 318, 262, 10299, 33955, 11, 257, 37919, 1295, 286, 11443, 290, 14580, 13, 632, 318, 257, 30069, 286, 262, 7128, 33955, 379, 406, 454, 8906, 11, 4881, 810, 262, 5283, 5335, 1128, 7241, 306, 4120, 284, 9281, 6206, 324, 5857, 311, 12944, 343, 516, 287, 1248, 3365, 13, 1629, 262, 886, 286, 262, 1388, 3708, 357, 392, 287, 257, 1277, 1627, 326, 20417, 832, 513, 25827, 290, 262, 3561, 31390, 828, 318, 257, 2829, 11, 3660, 7815, 15207, 286, 5335, 13], [19895, 5712, 20221, 11, 262, 1524, 468, 257, 7835, 2095, 13, 1629, 404, 262, 8774, 11819, 338, 386

In [11]:
token = [tokenizer(data, return_tensors="pt").input_ids for data in sample_dataset]
print(token)

[tensor([[19895,  5712, 20221,    11,   262,  1524,   468,   257,  7835,  2095,
            13,  1629,   404,   262,  8774, 11819,   338,  3869, 29500,   318,
           257, 10861, 15207,   286,   262,  5283,  5335,    13, 34528,   287,
          2166,   286,   262,  8774, 11819,   290,  6476,   340,    11,   318,
           257, 15317, 15207,   286,  1951,   351,  5101,   510, 49309,   351,
           262,  8177,   366, 37522,   578,  1215,  2185, 16543,  2516,  1911,
          7406,   284,   262,  8774, 11819,   318,   262, 32520,  3970,   286,
           262, 17380,  8894,    13, 34528,  2157,   262, 37792,  3970,   318,
           262, 10299, 33955,    11,   257, 37919,  1295,   286, 11443,   290,
         14580,    13,   632,   318,   257, 30069,   286,   262,  7128, 33955,
           379,   406,   454,  8906,    11,  4881,   810,   262,  5283,  5335,
          1128,  7241,   306,  4120,   284,  9281,  6206,   324,  5857,   311,
         12944,   343,   516,   287,  1248,  3365, 

In [12]:
token = [tokenizer(data, return_tensors="pt").input_ids[0] for data in sample_dataset]
print(token)

[tensor([19895,  5712, 20221,    11,   262,  1524,   468,   257,  7835,  2095,
           13,  1629,   404,   262,  8774, 11819,   338,  3869, 29500,   318,
          257, 10861, 15207,   286,   262,  5283,  5335,    13, 34528,   287,
         2166,   286,   262,  8774, 11819,   290,  6476,   340,    11,   318,
          257, 15317, 15207,   286,  1951,   351,  5101,   510, 49309,   351,
          262,  8177,   366, 37522,   578,  1215,  2185, 16543,  2516,  1911,
         7406,   284,   262,  8774, 11819,   318,   262, 32520,  3970,   286,
          262, 17380,  8894,    13, 34528,  2157,   262, 37792,  3970,   318,
          262, 10299, 33955,    11,   257, 37919,  1295,   286, 11443,   290,
        14580,    13,   632,   318,   257, 30069,   286,   262,  7128, 33955,
          379,   406,   454,  8906,    11,  4881,   810,   262,  5283,  5335,
         1128,  7241,   306,  4120,   284,  9281,  6206,   324,  5857,   311,
        12944,   343,   516,   287,  1248,  3365,    13,  1629,

In [13]:
batch_size, train_steps = 16, 300
train_samples = batch_size * train_steps

datasets = load_dataset("squad").data["train"]["context"]
datasets = [str(sample) for i, sample in enumerate(datasets) if i < train_samples]
datasets = [
        tokenizer(data, return_tensors="pt", max_length=1024).input_ids[0]
        for data in tqdm(datasets)
]

100%|██████████| 4800/4800 [00:02<00:00, 1809.30it/s]


In [14]:
datasets[:2]

[tensor([19895,  5712, 20221,    11,   262,  1524,   468,   257,  7835,  2095,
            13,  1629,   404,   262,  8774, 11819,   338,  3869, 29500,   318,
           257, 10861, 15207,   286,   262,  5283,  5335,    13, 34528,   287,
          2166,   286,   262,  8774, 11819,   290,  6476,   340,    11,   318,
           257, 15317, 15207,   286,  1951,   351,  5101,   510, 49309,   351,
           262,  8177,   366, 37522,   578,  1215,  2185, 16543,  2516,  1911,
          7406,   284,   262,  8774, 11819,   318,   262, 32520,  3970,   286,
           262, 17380,  8894,    13, 34528,  2157,   262, 37792,  3970,   318,
           262, 10299, 33955,    11,   257, 37919,  1295,   286, 11443,   290,
         14580,    13,   632,   318,   257, 30069,   286,   262,  7128, 33955,
           379,   406,   454,  8906,    11,  4881,   810,   262,  5283,  5335,
          1128,  7241,   306,  4120,   284,  9281,  6206,   324,  5857,   311,
         12944,   343,   516,   287,  1248,  3365,  

In [15]:
sample = [{"input_ids": data} for data in datasets[:4]]
print(type(sample))
print(sample)

<class 'list'>
[{'input_ids': tensor([19895,  5712, 20221,    11,   262,  1524,   468,   257,  7835,  2095,
           13,  1629,   404,   262,  8774, 11819,   338,  3869, 29500,   318,
          257, 10861, 15207,   286,   262,  5283,  5335,    13, 34528,   287,
         2166,   286,   262,  8774, 11819,   290,  6476,   340,    11,   318,
          257, 15317, 15207,   286,  1951,   351,  5101,   510, 49309,   351,
          262,  8177,   366, 37522,   578,  1215,  2185, 16543,  2516,  1911,
         7406,   284,   262,  8774, 11819,   318,   262, 32520,  3970,   286,
          262, 17380,  8894,    13, 34528,  2157,   262, 37792,  3970,   318,
          262, 10299, 33955,    11,   257, 37919,  1295,   286, 11443,   290,
        14580,    13,   632,   318,   257, 30069,   286,   262,  7128, 33955,
          379,   406,   454,  8906,    11,  4881,   810,   262,  5283,  5335,
         1128,  7241,   306,  4120,   284,  9281,  6206,   324,  5857,   311,
        12944,   343,   516,   287

In [16]:
tokenizer.pad(sample)

{'input_ids': tensor([[19895,  5712, 20221,    11,   262,  1524,   468,   257,  7835,  2095,
            13,  1629,   404,   262,  8774, 11819,   338,  3869, 29500,   318,
           257, 10861, 15207,   286,   262,  5283,  5335,    13, 34528,   287,
          2166,   286,   262,  8774, 11819,   290,  6476,   340,    11,   318,
           257, 15317, 15207,   286,  1951,   351,  5101,   510, 49309,   351,
           262,  8177,   366, 37522,   578,  1215,  2185, 16543,  2516,  1911,
          7406,   284,   262,  8774, 11819,   318,   262, 32520,  3970,   286,
           262, 17380,  8894,    13, 34528,  2157,   262, 37792,  3970,   318,
           262, 10299, 33955,    11,   257, 37919,  1295,   286, 11443,   290,
         14580,    13,   632,   318,   257, 30069,   286,   262,  7128, 33955,
           379,   406,   454,  8906,    11,  4881,   810,   262,  5283,  5335,
          1128,  7241,   306,  4120,   284,  9281,  6206,   324,  5857,   311,
         12944,   343,   516,   287,  

In [17]:
tokenizer.pad(sample, padding="max_length")

{'input_ids': tensor([[19895,  5712, 20221,  ..., 50256, 50256, 50256],
        [19895,  5712, 20221,  ..., 50256, 50256, 50256],
        [19895,  5712, 20221,  ..., 50256, 50256, 50256],
        [19895,  5712, 20221,  ..., 50256, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [18]:
batch_size, train_steps = 16, 300
train_samples = batch_size * train_steps

datasets = load_dataset("squad").data["train"]["context"]
datasets = [str(sample) for i, sample in enumerate(datasets) if i < train_samples]
datasets = [
        tokenizer(data, return_tensors="pt", max_length=1024).input_ids[0]
        for data in tqdm(datasets)
]

def collate_fn(batch):
    batch_encoding = tokenizer.pad(
        {"input_ids": batch}, padding="max_length", max_length=1024
    )
    return batch_encoding.input_ids


data_loader = DataLoader(
            #datasets,
            sorted(datasets, key=len, reverse=True),
            # uniform length batching
            # https://mccormickml.com/2020/07/29/smart-batching-tutorial/
            batch_size=batch_size,
            num_workers=8,
            collate_fn=collate_fn,
            shuffle=False,
        )

100%|██████████| 4800/4800 [00:02<00:00, 1807.84it/s]


In [19]:
for batch in data_loader:
    break

In [20]:
[len(x) for x in batch]

[1024,
 1024,
 1024,
 1024,
 1024,
 1024,
 1024,
 1024,
 1024,
 1024,
 1024,
 1024,
 1024,
 1024,
 1024,
 1024]

In [21]:
batch[:2]

tensor([[41000, 45436,   286,  ..., 50256, 50256, 50256],
        [41000, 45436,   286,  ..., 50256, 50256, 50256]])

In [90]:
from transformers import DataCollatorWithPadding, get_scheduler

batch_size = 2
datasets = load_dataset("squad")
      
tokenized_datasets = datasets.map(lambda x: tokenizer(x["context"], truncation=True, padding="max_length"), batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(datasets["train"].column_names)
#data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def data_collator(batch):
    print(type(batch))
    a = [x["input_ids"] for x in batch]
    return torch.tensor(a)

data_loader = DataLoader(
         tokenized_datasets["train"], shuffle=False, batch_size=batch_size, collate_fn=data_collator
)


In [91]:
for batch in data_loader:
    break
print(batch)

<class 'list'>
tensor([[19895,  5712, 20221,  ..., 50256, 50256, 50256],
        [19895,  5712, 20221,  ..., 50256, 50256, 50256]])


In [96]:
from transformers import DataCollatorWithPadding, get_scheduler

batch_size = 16
datasets = load_dataset("squad")
      
tokenized_datasets = datasets.map(lambda x: tokenizer(x["context"], truncation=True), batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(datasets["train"].column_names)

#data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def data_collator(batch):
    batch_encoding = tokenizer.pad(
        #{"input_ids": batch}, return_tensors="pt", padding="max_length", max_length=1024
        {"input_ids": batch}, return_tensors="pt", padding=True
    )
    return batch_encoding.input_ids

    
data_loader = DataLoader(
         tokenized_datasets["train"]["input_ids"], shuffle=False, batch_size=batch_size, collate_fn=data_collator
)


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [97]:
for batch in data_loader:
    break
print(batch.shape)

torch.Size([16, 261])


In [45]:
tokenized_datasets["train"]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 87599
})

In [28]:
from transformers import DataCollatorWithPadding, get_scheduler

batch_size = 16
datasets = load_dataset("squad")
      
tokenized_datasets = datasets.map(lambda x: tokenizer(x["context"], truncation=True, padding="max_length"), batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(datasets["train"].column_names)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_loader = DataLoader(
         tokenized_datasets["train"], shuffle=False, batch_size=batch_size, collate_fn=data_collator
)


In [27]:
sorted_datasets = sorted(tokenized_datasets["train"]["input_ids"], key=len, reverse=True)
#print([len(x) for x in sorted_datasets])

In [33]:
from transformers import DataCollatorWithPadding, get_scheduler

batch_size = 16
train_datasets = load_dataset("squad", split="train")
tokenized_datasets = tokenizer(train_datasets["context"], truncation=True)
#sorted_datasets = sorted(tokenized_datasets["input_ids"], key=len, reverse=True)


In [35]:
print(len(tokenized_datasets))
print(tokenized_datasets[:2])

2
{'input_ids': [[19895, 5712, 20221, 11, 262, 1524, 468, 257, 7835, 2095, 13, 1629, 404, 262, 8774, 11819, 338, 3869, 29500, 318, 257, 10861, 15207, 286, 262, 5283, 5335, 13, 34528, 287, 2166, 286, 262, 8774, 11819, 290, 6476, 340, 11, 318, 257, 15317, 15207, 286, 1951, 351, 5101, 510, 49309, 351, 262, 8177, 366, 37522, 578, 1215, 2185, 16543, 2516, 1911, 7406, 284, 262, 8774, 11819, 318, 262, 32520, 3970, 286, 262, 17380, 8894, 13, 34528, 2157, 262, 37792, 3970, 318, 262, 10299, 33955, 11, 257, 37919, 1295, 286, 11443, 290, 14580, 13, 632, 318, 257, 30069, 286, 262, 7128, 33955, 379, 406, 454, 8906, 11, 4881, 810, 262, 5283, 5335, 1128, 7241, 306, 4120, 284, 9281, 6206, 324, 5857, 311, 12944, 343, 516, 287, 1248, 3365, 13, 1629, 262, 886, 286, 262, 1388, 3708, 357, 392, 287, 257, 1277, 1627, 326, 20417, 832, 513, 25827, 290, 262, 3561, 31390, 828, 318, 257, 2829, 11, 3660, 7815, 15207, 286, 5335, 13], [19895, 5712, 20221, 11, 262, 1524, 468, 257, 7835, 2095, 13, 1629, 404, 262, 8774,

In [39]:
batch_size = 24
datasets = load_dataset("squad")
def tokenize_function(element):
        return tokenizer(element["context"], truncation=True)

#tokenized_datasets = datasets.map(tokenize_function, batched=True)
tokenized_datasets = datasets.map(lambda x: tokenizer(x["context"], truncation=True), batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(datasets["train"].column_names)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

data_loader = DataLoader(
    tokenized_datasets["train"], shuffle=False, batch_size=batch_size, collate_fn=data_collator
)


In [41]:
for batch in data_loader:
    break
batch["input_ids"].shape

torch.Size([24, 261])